In [1]:
%autosave 120


Autosaving every 120 seconds


In [1]:
import pandas as pd
import xlrd
import numpy as np
import openpyxl as opx
import glob
import sys
import datetime
import xlwings as xl
import copy
from openpyxl.utils import range_boundaries
from fuzzywuzzy import fuzz
import pyodbc
import math as math
path = 'C:\\trial'
folder = glob.glob(path + '\\*')
company = [1]
c = 0

C:\Users\admin\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#connection to database:
server = 'yatcru9wf3.database.windows.net'
database = 'BA'
username = 'kagadmin'
password = '#+E5tANDu5e2k18#'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [3]:

data = []
#function which get information of file using its path and return 5 elements(year, month,date,ticker code,analyst name)
def formating(filen):
    data = filen.replace(path, '').split('\\')
    del data[0]
    del data[3]
    if '-' in data[-1]:
        temp = (data[-1].split('.')[0]).split('-')
    elif '_' in data[-1]:
        temp = (data[-1].split('.')[0]).split('_')
    else:
        temp = (data[-1].split('.')[0]).split(' ')
    del data[-1]
    data.append(temp[0])
    data.append(temp[1])
    print(data)
    return(data)

In [4]:

#defined variable which help us to recognize year and quater of given column

year_pattern_int=[]
year_pattern_float=[]
year_pattern_2di=[]
quater_pattern=['q1','1q','2q','q2','q3','3q','4q','q4']
bscf_names=["balancesheet & cashflow","cash & balance","adjusted bs","annual bs&cf","income statement"]
#bscf_names=["balancesheet","cashflows","bs&cf","cf&bs","bsheet","balancesheet&cashflow","cashflow&balance sheet","cash&balance","adjustedbs","annualbs&cf","annualcf&bs"]

check_list = ['Q1', 'Q2', 'Q3', 'Q4', '1Q', '2Q', '3Q', '4Q',1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
             2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
             2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035,'FY','CY']
for i in range(2000,2031):
    year_pattern_int.append(i)
    year_pattern_float.append(float(i))
    if i-2000<10:
        year_pattern_2di.append('0'+str(i-2000))
    else:
        year_pattern_2di.append(str(i-2000))


In [30]:
# type of main function

def function1(excel_file,path_list):
    #read excel file using openpyxl
    wb=opx.load_workbook(excel_file,data_only=True)
    sheet_tobechecked=wb.sheetnames #getting all the sheet name
    nonhidden=[]
    startrow={}
    print(sheet_tobechecked)
    #this part helps to ignore hidden sheets and to take related to BS , Cf and IS
    names=[]
    for i in range(0, len(sheet_tobechecked)):
        curr_sheet = wb[sheet_tobechecked[i]]
        if curr_sheet.sheet_state != 'hidden':
            nonhidden.append(sheet_tobechecked[i])
            if sheet_tobechecked[i].lower()=='bs' or sheet_tobechecked[i].lower()=='cf' or sheet_tobechecked[i].lower()=='model':
                names.append(sheet_tobechecked[i])
            else:
                count_nobscf=1
                for nam in bscf_names:
                    if fuzz.partial_token_set_ratio(sheet_tobechecked[i].lower(),nam)>=80:
                        names.append(sheet_tobechecked[i])
    
    sheet_tobechecked = names # all the sheet which are important to us
    
    #if no sheet found then return giving error as sheet_name_fault
    if len(sheet_tobechecked)<=0:
        return (2) # that means there is problem
        '''print(nonhidden)
        while True:
            sheee=input("Enter the sheet name U want to process from above or for exit input'e' ")
            if sheee!='e':
                sheet_tobechecked.append(sheee)
            else:
                break'''
    
    sheet_tobechecked=list(set(sheet_tobechecked))# to remove repeated sheet names
    print(sheet_tobechecked)
    print("sheet to be checked")
        
    #defining variable
    year_pre,quater_pre,q4_check_pre='','',0
    
    #unmerging all cells of every sheet of a file
    for every_sheet in sheet_tobechecked:
        
        startrow_no=startingrow(wb,every_sheet)-1
        if startrow_no<0:
            startrow[every_sheet]=0
        else:
            startrow[every_sheet]=startrow_no
        
        curr_sheet=wb[every_sheet]
        merge_list=copy.deepcopy(curr_sheet.merged_cells.ranges)
        for group in merge_list :
            split_arr = group.coord.split(':')
            min_col, min_row, max_col, max_row = range_boundaries(group.coord)
            top_left_cell_value = curr_sheet[split_arr[0]].value
            curr_sheet.unmerge_cells(group.coord)
            for row in curr_sheet.iter_rows(min_col=min_col, min_row=min_row, max_col=max_col, max_row=max_row):
                for cell in row:
                    cell.value = top_left_cell_value
    wb.save(excel_file) #the modified file is been overwritten (have to look over it afterward by changing path)
    print("excel")
    display(excel_file)
    for sheetno in range(0,len(sheet_tobechecked)):
        print(sheet_tobechecked[sheetno])
        address_dataframe=pd.DataFrame(columns=['q1','q2','q3','q4','tl','Ex']) # defining lookup table
        excel_data = pd.read_excel(excel_file,sheet_name=sheet_tobechecked[sheetno],header=None,skiprows=[i for i in range(0,startrow[sheet_tobechecked[sheetno]])])
        pd.options.display.max_columns = None
        pd.options.display.max_rows=None
        print("This is excel data")
        display(excel_data)
        
        #formating the dataframe by removing column and row having less than 10 element
        excel_data=excel_data.replace(0,np.nan)
        excel_data=excel_data.dropna(axis=0,thresh=10)
        excel_data=excel_data.dropna(axis=1,thresh=10)
        excel_data=excel_data.reset_index(drop=True)
        if(len(excel_data)==0):
            continue
        excel_data.columns = range(excel_data.shape[1])
        pd.options.display.max_columns = None
        pd.options.display.max_rows=None
        display(excel_data)
        startrowno=0
        
        #getting defination of each column
        print(len(excel_data.columns))
        if len(excel_data.columns)>10: #changed from 10 to 3
            temp_dataframe=excel_data[:10] # the first 10 element of a column will tell us about it 
            for j in temp_dataframe:
                list_ele=list(temp_dataframe[j])
                knowledge=[]
                colno=j
                for i in list_ele:
                    #overlook element if its nan,none, or weeks or days written in it
                    if str(i)=='nan' or str(i)=='None' or type(i)==datetime.datetime or "week" in str(i) or "day" in str(i):
                        a=1
                    #take only element which are keywords to us (like FY2017,3Q17 etc.) or else break which means any value have been occured
                    elif (type(i)==float and i not in year_pattern_float) or (type(i)==int and i not in year_pattern_int):
                        startrowno=list_ele.index(i)
                        break
                    else:
                        knowledge.append(i)
                
                # proces further if the list taken for defining the row is not empty 
                if len(knowledge)>0:
                    
                    year_got,quater_got,q4_check_got=about_q_year(knowledge)
                        
                    #here is a small assumtion (if we got Q4 and all the elementused to define column is same we will take the curent row as Total of particuar year) )
                    if q4_check_got==1 and year_got==year_pre and quater_got==quater_pre and q4_check_got==q4_check_pre:
                        year_final=year_got
                        quater_final='TL'
                        q4_check_final=1
                    else:
                        # or else everything will be consider as it was given has been found about it
                        year_pre=year_got
                        quater_pre=quater_got
                        q4_check_pre=q4_check_got
                        year_final=year_got
                        quater_final=quater_got
                        q4_check_final=q4_check_got                       
                    
                    #if only year is written and nothing about the quater we take that as Total for particular year
                    if year_got!='' and (quater_got=='' or quater_got=='TL' or quater_pre=='TL' or quater_pre==''):
                        quater_final='TL'
                    
                    #printing and making dataframe(lookup table) of information of each columns
                    if year_final!='' and quater_final!='':
                        got_cor=1 #means we got something
                        #print(knowledge,"===",end="")
                        #print("belongs to "+"20"+year_final," "+quater_final)
                        if '1' in quater_final:
                            qval='q1'
                        elif '2' in quater_final:
                            qval='q2'
                        elif '3' in quater_final:
                            qval='q3'
                        elif '4' in quater_final:
                            qval='q4'
                        else:
                            qval='tl'
                        try:
                            address_dataframe.loc["20"+year_final][qval]=[startrowno,colno]
                        except Exception:
                            address_dataframe.loc["20"+year_final]='Nan'
                            address_dataframe.loc["20"+year_final][qval]=[startrowno,colno]                           
                 
        display(address_dataframe)
        #db_connect()
        #extra fro shoeing rows which doesnot match
        not_present=pd.DataFrame(columns=excel_data.columns)
        coll=0
        
        #this is to make a list of year and its respective column number from lookup take which has to consider
        row_name_year=list(address_dataframe.index)
        row_name_year.sort()
        if int(path_list[0])<int(row_name_year[-1]):
            cur_yr=int(path_list[0])
            count_yr=0
            count = 0
            metric_col=[]
            par_year=[]
            metric_row=20
            year_tosearch=cur_yr
            
            lowest_year=min_year("'"+path_list[3]+"'")

            while (count_yr<6 and year_tosearch>=lowest_year):
                year_tosearch=cur_yr-(count_yr+1)
                val=address_dataframe['q1'][str(year_tosearch)]
                if val!='Nan':
                    if val[0]<metric_row:
                        metric_row=val[0]
                    metric_col.append(val[1])
                    par_year.append(year_tosearch)
                    count_yr+=1
            print(par_year)
            print(metric_col)
            print(metric_row)
                

            all_metric=set()
            for rowss in range(metric_row,excel_data.shape[0]):
                metric_found=[]
                list_metric_found=[]
                com_met=set()
                for met_num in range(0,6):
                    #getting the metric name wrt the year,quater,ticker and value send
                    metric=set(get_metric_name(par_year[met_num],path_list[3],excel_data.iloc[rowss][metric_col[met_num]]))
                    com_met.update(metric)
                    list_metric_found.extend(metric)
                if com_met:
                    print(com_met)
                    print(list_metric_found)
                for un_met in com_met:
                    #if min 60% of them says that yes the values are of  particlar metric we take it 
                    if list_metric_found.count(un_met)/6 >= 0.6:
                        metric_found.append(un_met)
                if metric_found:
                    print(rowss,metric_found)
                    all_metric.update(metric_found)
                else:
                    not_present.loc[coll]=list(excel_data.loc[rowss])
                    coll+=1
                
            print(all_metric,len(all_metric))#printing all mertric found in database 
            
            metric_db=get_metriclist_from_db(path_list[0],path_list[3])
            print("metric not in actuals---")
            print(list(set(metric_db)-set(all_metric)))# all metric we couldn't find wrt to a ticker
            print("metric not with respect to analyst")
            print(list(set(get_metriclist_wrt_analyst(path_list[3],path_list[4]))-set(all_metric)))#metric which we couldn't finf wrt to analyst
            # the percentge accuracy will be calculate wby no of metric_found by us divided by total number of metric wrt analyst 
            print()
            print()
            print("ACCURACY==")
            print("Number of total element in Actual DB=", len(metric_db))
            print("Number of metric found by us=",len(all_metric))

            print("accuracy wrt to Actual=",len(list(set(metric_db)&set(all_metric)))*100/len(metric_db))
            print()
            print("Number of total element in Estimated DB wrt Analyst=", len(get_metriclist_wrt_analyst(path_list[3],path_list[4])))
            z=set(get_metriclist_wrt_analyst(path_list[3],path_list[4]))
            print("Number of metric found by us=",len(z&set(all_metric))*100/len(z))

            
            display(not_present)
    
    if got_cor:
        return 0
    else:
        return 1
   
    '''qw=input()
            if qw=='q':
                sys.exit()'''


In [6]:

#get the minimum year present in dtabase for given ticker
def min_year(ticker):
    select_stmt1 = F" select distinct financial_year from dbo.cc_actual_metrics_consolidated where ticker_code={ticker} order by financial_year"
    cursor.execute(select_stmt1)
    row = cursor.fetchone()
    res_array = []
    while row:
        res_array.append(row[0])
        row = cursor.fetchone()
    print(res_array[0])
    return(res_array[0])


In [7]:
def get_metric_name (year,ticker,value,quarter='Q1'):
    ticker_code = "'"+ticker+"'"
    year = year
    quarter = "'"+quarter+"'"

    if type(value)==int or (type(value)==float and str(value)!='nan'):
        if abs(value)>=1:
            #when value is greater than 1
            valueo= "'"+str(value)+"'" #orginal value
            value1 = "'"+str(int(value))+".%"+"'" # integered value
            value2 = "'"+str(round(value, 2))+"%"+"'" #value when rounded upto 2
            value3 = "'"+str(-1*int(value))+".%"+"'" #integered with negation
            value4 = "'"+str(-1*round(value,2))+"%"+"'" # round upto 2 with negation
            print(valueo,value1,value2,value3,value4)
            select_stmt1 = F"select distinct metric_name from dbo.cc_actual_metrics_consolidated where " \
                F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                F"(value like {valueo} or value like {value1} or value like {value2} or value like {value3} or value like {value4})"

            metric_list=[]
            cursor.execute(select_stmt1)
            row = cursor.fetchone()
            while row:
                
                metric_list.append(row[0])
                row = cursor.fetchone()
        else:
            #if value less than 1
            valueo= "'"+str(value*100)+"'" #orginal with 100 multiplied
            value1 = "'"+str(int(value*100))+".%"+"'" # multiply by 100 and then integered
            value3 = "'"+str(-1*int(value*100))+".%"+"'" # multiply by 100 and integer and negate it
            print("with 100",valueo,value1,value3)
            select_stmt1 = F"select distinct metric_name from dbo.cc_actual_metrics_consolidated where " \
                F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                F"(value like {valueo} or value like {value1} or value like {value3})"
            
            metric_list=[]
            cursor.execute(select_stmt1)
            row = cursor.fetchone()
            flag_got_with_100=0
            while row:
                metric_list.append(row[0])
                row = cursor.fetchone()
                flag_got_with_100=1

            #if not flag_got_with_100:
            valueo = "'"+str(value)+"'" #orginal value
            value1 = "'"+str(round(value,2))+"%"+"'" # round upto 2
            value3 = "'"+str(-1*round(value,2))+"%"+"'" # round upto 2  and negate it
            print("without 100",valueo,value1,value3)
            select_stmt1 = F"select distinct metric_name from dbo.cc_actual_metrics_consolidated where " \
                F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                F"(value like {valueo} or value like {value1} or value like {value3})"
            
            cursor.execute(select_stmt1)
            row = cursor.fetchone()
            while row:
                #to_confirm
                metric_list.append(row[0])
                row = cursor.fetchone()

        return(metric_list)
    else:
        return([])


In [8]:
#function which tells about the quater and year the particular column belongs to
def about_q_year(checkar):
    
    checkar_duplicate=np.copy(checkar)
    quater=''
    year=''
    year_flag=0
    quater_flag=0
    q4_check=0
    checkar=[]
    datetime_flag=1
    for i_che in checkar_duplicate:
        checkar.append(str(i_che))
    
    for i_che in range(0,len(checkar)):
        
        if not year_flag:
            #checking if year is presenr in (2017,2016 format)
            for j_y in year_pattern_int:
                if str(j_y) in checkar[i_che]:
                    year=str(j_y)[2:]
                    checkar[i_che]=checkar[i_che].replace(str(j_y),'')
                    checkar[i_che]=checkar[i_che].replace(year,'')
                    year_flag=1
                    break
        if not year_flag:
            #checking if the year is present in 2 digited format like 17,16)
            for j_y in year_pattern_2di:
                if str(j_y) in checkar[i_che]:
                    year=str(j_y)
                    checkar[i_che]=checkar[i_che].replace(year,'')
                    year_flag=1
                    break

        if not quater_flag:
            # getting info about the quater
            for i_p in quater_pattern:
                checkar[i_che]=checkar[i_che].replace(year,'')
                checkar[i_che]=checkar[i_che].replace("20"+year,'')
                if i_p in checkar[i_che].lower():
                    quater=i_p
                    quater_flag=1
                    break       
        
        if year_flag and quater_flag:
            break
    #returning the year and quater and if its 4th quatered or not
    return(year,quater,q4_check)


In [9]:
# funtion to avoid te rough work present in top and retutn the row no from which its important
def startingrow(wb,sheet_name):
    curr_sheet = wb[sheet_name]
    flag = 0
    row = 0
    for col in curr_sheet.iter_cols(max_col=20,max_row=15):
        for cell in col:
            if cell.value is not None:
                for data in check_list:
                    if fuzz.partial_ratio(str(data),str(cell.value)) == 100:
                        row = cell.row
                        flag =1
                        break
                if flag == 1:
                    break
        if flag == 1:
            break

    print(row)
    return(row)

In [10]:
#function which return all the metric anme with repect to ticker in actuals
def get_metriclist_from_db(year, ticker):

    ticker_code = "'"+ticker+"'"
    year = year
    select_stmt1 = F"select distinct metric_name from dbo.cc_actual_metrics_consolidated where ticker_code = {ticker_code} " \
        F" and financial_year = {year}"

    cursor.execute(select_stmt1)
    row = cursor.fetchone()
    metric_list_comp = []
    while row:
        metric_list_comp.append(row[0])
        row = cursor.fetchone()

    return (metric_list_comp)

In [11]:
#function which return aall the metric name wrt to analyst from estimated database
def get_metriclist_wrt_analyst(ticker,analyst):
    ticker_code = "'"+ticker+"'"
    analyst_name="'%"+analyst+"%'"
    select_stmt1 = F" select distinct metric_name from [dbo].[cc_metrics_consolidated] where ticker_code = {ticker_code} and quarter='q1' and analyst_name like {analyst_name} order by metric_name"
    cursor.execute(select_stmt1)
    row = cursor.fetchone()
    res_array = []
    while row:
        res_array.append(row[0])
        row = cursor.fetchone()
    return(res_array)

In [31]:
filen="C:\\trial\\2018\\may\\2 MAY\\lvs\\LVS-Santarelli.xlsx"
print(filen)
path_list=formating(filen)
val=function1(filen,path_list)
if val==1:
    print("FAULT")
elif val==2:
    print("fault with sheet name")

C:\trial\2018\may\2 MAY\lvs\LVS-Santarelli.xlsx
['2018', 'may', '2 MAY', 'LVS', 'Santarelli']
['__FDSCACHE__', 'Earnings Model', 'US Revenue & EBITDA Drivers', 'Macau Revenue & EBITDA Drivers', 'Macau Aggregator Model', 'MBS Revenue & EBITDA Drivers', 'FCF & Balance Sheet Model', 'Sands China Model', 'Price Target', 'PT Composition', 'Target Sensitivity Parts', 'Company Tabs----->', 'LVS Adjusted Valuation', 'Historical Valuation', 'EV-EBITDA Valuation Charts', 'Comparative Valuation', 'ROIC', 'Dividend History', 'Estimate Summary', 'Estimate Assumptions', 'FCF Waterfall', 'FCF Parts', 'Free Cash Flow', 'Summary of Estimate Changes', 'Development Pipeline', '3Q18 Comparison', '3Q18 Property-Level Comparison', 'Consensus', 'Preview Consensus Comparison', 'Y-Y & Seq. Preview Comparison', 'Prop Preview Comparison', 'Balance Sheet Stats', 'Hold Tracker', 'Current Assets', 'EBITDA Mix', 'EBITDA Mix Charts', 'Provision for Doubtful Accts', 'Macau Related Tabs ----->', 'Venetian Macau Propert

'C:\\trial\\2018\\may\\2 MAY\\lvs\\LVS-Santarelli.xlsx'

Balance Sheet Stats
This is excel data


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,NaN,NaN,2009.000000,NaN,2010.000000,NaN,2011.000000,NaN,2012.000000,NaN,2013.000000,NaN,2014.000000,NaN,2015.000000,NaN,2016.000000,NaN,2017.000000,NaN,2018E,NaN,2019E,NaN,2020E
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cash,NaN,5074.057000,NaN,3847.001000,NaN,3907.546000,NaN,2512.766000,NaN,3600.414000,NaN,3512.900000,NaN,2187.391000,NaN,2137.000000,NaN,2430.000000,NaN,4785,NaN,4785,NaN,4785
3,Gross Debt,NaN,11025.462000,NaN,10140.823000,NaN,10032.977000,NaN,10230.067000,NaN,9760.257000,NaN,9992.647000,NaN,9468.012000,NaN,9595.000000,NaN,9640.000000,NaN,11694.8,NaN,10787.5,NaN,10680.8
4,Net Debt,NaN,5951.405000,NaN,6293.822000,NaN,6125.431000,NaN,7717.301000,NaN,6159.843000,NaN,6479.747000,NaN,7280.621000,NaN,7458.000000,NaN,7210.000000,NaN,6909.81,NaN,6002.48,NaN,5895.85
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Interest Expense,NaN,321.870000,NaN,306.813000,NaN,282.949000,NaN,258.564000,NaN,271.211000,NaN,274.181000,NaN,265.220000,NaN,273.874000,NaN,327.000000,NaN,438.7,NaN,487.7,NaN,470.8
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,EBITDA,NaN,924.164000,NaN,2078.423000,NaN,3303.095000,NaN,3543.867000,NaN,4533.468000,NaN,5206.995000,NaN,3955.416000,NaN,3836.540000,NaN,4689.000000,NaN,5137.3,NaN,5257.71,NaN,5629.12
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


9


""
0
1
2
3
4
5
6
7
8


0


,q1,q2,q3,q4,tl,Ex


IndexError: list index out of range

In [ ]:
!pip install 

In [18]:
path_list[0]

'2018'

In [19]:
row_name_year[-1]

NameError: name 'row_name_year' is not defined

In [20]:
address_dataframe


NameError: name 'address_dataframe' is not defined

NameError: name 'excel_data' is not defined